# * VINSIGHT : Metric Insight

## Parameter

In [175]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [176]:
''' Parameter Input '''

v_metric_cd = 'B0R000100GEO' #Total Revenue (Geo)
# v_metric_cd = 'B0R00010001CS' #Total Inflow M1

# v_start_date = 20240101

print(f'\nParameter input...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_start_date: {v_start_date}')


Parameter input...

   -> v_metric_cd: B0R000100GEO


In [177]:
''' Automate Month Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

## Metric Insight

In [178]:
''' Execute transaction '''

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
        , ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH, ACTUAL_AGG_QTR, ACTUAL_AGG_YR 
        , TARGET_SNAP, TARGET_AGG_MTH, TARGET_AGG_QTR, TARGET_AGG_YR 
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = '{v_metric_cd}'
    --AND TM_KEY_DAY >= {v_start_date}
    --ORDER BY TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Data as of 2025-05-29, 14:07:00



   -> DataFrame : 109501 rows, 21 columns


### Overview

In [179]:
''' ACTUAL : Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df['P_TARGET'] = np.where(monthly_df['AREA_CD']=='P', monthly_df['TARGET_SNAP'], 0)
monthly_df['C'] = np.where(monthly_df['AREA_CD']=='C', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['P'] = np.where(monthly_df['AREA_CD']=='P', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['G'] = np.where(monthly_df['AREA_TYPE']=='G', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['H'] = np.where(monthly_df['AREA_TYPE']=='H', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['HH'] = np.where(monthly_df['AREA_TYPE']=='HH', monthly_df['ACTUAL_SNAP'], 0)

monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

monthly_df_display

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P_TARGET,C,P,G,H,HH
0,202303,All Services,B0R000100GEO,Total Revenue (Geo),2024-04-23 12:22:16,20230331.0,"11,760,976,279",0,"11,473,254,492","11,444,123,608","5,498,940,307",0
1,202304,All Services,B0R000100GEO,Total Revenue (Geo),2024-04-23 12:22:16,20230430.0,"11,690,488,772",0,"11,405,931,856","11,376,427,038","5,455,100,467",0
2,202305,All Services,B0R000100GEO,Total Revenue (Geo),2024-04-23 12:22:16,20230531.0,"11,720,232,395",0,"11,509,592,904","11,480,138,858","5,500,190,589",0
3,202306,All Services,B0R000100GEO,Total Revenue (Geo),2024-04-23 12:22:16,20230630.0,"11,723,274,957",0,"11,404,604,123","11,376,783,747","5,460,226,723",0
4,202307,All Services,B0R000100GEO,Total Revenue (Geo),2024-04-23 12:22:16,20230731.0,"11,732,696,627",0,"11,587,363,977","11,560,197,096","5,483,975,283",0
5,202308,All Services,B0R000100GEO,Total Revenue (Geo),2024-04-23 12:22:16,20230831.0,"11,752,522,262",0,"11,620,432,809","11,590,888,694","5,581,509,959",0
6,202309,All Services,B0R000100GEO,Total Revenue (Geo),2024-04-23 12:22:16,20230930.0,"11,670,824,267",0,"11,448,335,066","11,418,825,891","5,493,226,172",0
7,202310,All Services,B0R000100GEO,Total Revenue (Geo),2024-04-23 12:22:16,20231031.0,"11,749,830,624",0,"11,640,271,269","11,601,441,415","5,612,077,369",0
8,202311,All Services,B0R000100GEO,Total Revenue (Geo),2024-04-23 12:22:16,20231130.0,"11,668,843,196",0,"11,846,008,925","11,813,944,392","5,680,409,118",0
9,202312,All Services,B0R000100GEO,Total Revenue (Geo),2024-04-23 12:22:16,20231231.0,"11,860,365,583",0,"12,058,659,864","12,013,795,964","5,760,961,166",0


In [180]:
''' ACTUAL : Daily Summary '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

daily_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==v_tm_key_mth].copy()
daily_df['P_TARGET'] = np.where(daily_df['AREA_CD']=='P', daily_df['TARGET_SNAP'], 0)
daily_df['C'] = np.where(daily_df['AREA_CD']=='C', daily_df['ACTUAL_SNAP'], 0)
daily_df['P'] = np.where(daily_df['AREA_CD']=='P', daily_df['ACTUAL_SNAP'], 0)
daily_df['G'] = np.where(daily_df['AREA_TYPE']=='G', daily_df['ACTUAL_SNAP'], 0)
daily_df['H'] = np.where(daily_df['AREA_TYPE']=='H', daily_df['ACTUAL_SNAP'], 0)
daily_df['HH'] = np.where(daily_df['AREA_TYPE']=='HH', daily_df['ACTUAL_SNAP'], 0)

daily_df = daily_df.groupby(['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
daily_df = daily_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

daily_df_display = daily_df.copy()
mod_col_list = daily_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    daily_df_display[col] = daily_df_display[col].apply(lambda x: format(x, ',.0f'))

daily_df_display

,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P_TARGET,C,P,G,H,HH
0,20250501,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"163,008,010",0,"181,113,461","181,033,445","181,030,959","181,030,959"
1,20250502,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"1,008,924,196",0,"992,937,702","992,843,881","992,779,866","992,748,908"
2,20250503,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"237,739,560",0,"233,704,819","233,601,093","233,600,041","233,600,041"
3,20250504,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"234,556,209",0,"226,737,538","226,659,507","226,657,985","226,657,985"
4,20250505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"232,503,537",0,"229,099,212","229,023,042","229,022,806","229,022,806"
5,20250506,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"654,334,672",0,"675,602,815","675,531,255","675,602,122","675,605,749"
6,20250507,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"229,125,586",0,"224,161,542","223,690,403","223,670,115","223,670,115"
7,20250508,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"227,633,493",0,"225,753,269","188,984,579","188,983,688","188,983,688"
8,20250509,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"225,067,859",0,"221,456,281","185,293,574","185,292,315","185,292,315"
9,20250510,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"966,711,241",0,"931,271,133","894,264,405","894,340,230","894,345,810"


### Area Summary

In [181]:
''' Prep Monthly Data '''

raw_monthly_df = chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
raw_monthly_df = raw_monthly_df[['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
raw_monthly_df['GAP'] = raw_monthly_df['ACTUAL_SNAP'] - raw_monthly_df['TARGET_SNAP']
raw_monthly_df['%ACH'] = raw_monthly_df['ACTUAL_SNAP'] / raw_monthly_df['TARGET_SNAP'] * 100
raw_monthly_df['%PORTION'] = raw_monthly_df['ACTUAL_SNAP'] / raw_monthly_df.groupby(['TM_KEY_MTH', 'AREA_TYPE'])['ACTUAL_SNAP'].transform('sum') * 100

# raw_monthly_df['%ACH'] = raw_monthly_df['%PORTION'].apply(lambda x: format(x, ',.2f'))
# raw_monthly_df['%PORTION'] = raw_monthly_df['%PORTION'].apply(lambda x: format(x, ',.2f'))

raw_monthly_df = raw_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD', 'AREA_TYPE', 'AREA_CD']).reset_index(drop=True)
# raw_monthly_df.tail()

In [182]:
''' Nationwide (P) : Monthly Summary '''

# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501
v_start_mth = 202501

# p_monthly_df = raw_monthly_df.copy()
p_monthly_df = raw_monthly_df.loc[raw_monthly_df['TM_KEY_MTH']>=v_start_mth].copy()
p_monthly_df = p_monthly_df.loc[p_monthly_df['AREA_TYPE']=='P']
p_monthly_df = p_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)

p_monthly_df_display = p_monthly_df.copy()
mod_col_list = p_monthly_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    p_monthly_df_display[col] = p_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
p_monthly_df_display['%ACH'] = p_monthly_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
p_monthly_df_display['%PORTION'] = p_monthly_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
p_monthly_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202501,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250131.0,P,P,Nationwide,"12,034,347,430","12,359,344,311","-324,996,881",97.37,100.00
1,202502,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250228.0,P,P,Nationwide,"11,619,121,062","11,965,339,956","-346,218,894",97.11,100.00
2,202503,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250331.0,P,P,Nationwide,"11,995,980,677","12,403,581,248","-407,600,571",96.71,100.00
3,202504,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250430.0,P,P,Nationwide,"11,891,585,207","12,190,349,530","-298,764,324",97.55,100.00
4,202505,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527.0,P,P,Nationwide,"10,398,151,045","10,771,445,001","-373,293,957",96.53,100.00


In [183]:
''' 8 Region (G) : Monthly Summary '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

g_monthly_df = raw_monthly_df.loc[raw_monthly_df['TM_KEY_MTH']==v_tm_key_mth].copy()
g_monthly_df = g_monthly_df.loc[g_monthly_df['AREA_TYPE']=='G']
# g_monthly_df = g_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)
g_monthly_df = g_monthly_df.fillna(0).sort_values(by='%ACH', ascending=False).reset_index(drop=True)

g_monthly_df_display = g_monthly_df.copy()
mod_col_list = g_monthly_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    g_monthly_df_display[col] = g_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
g_monthly_df_display['%ACH'] = g_monthly_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
g_monthly_df_display['%PORTION'] = g_monthly_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
g_monthly_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202505,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527.0,G,GX3,East,"1,803,950,098","1,799,689,709","4,260,388",100.24,17.66
1,202505,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527.0,G,GX1,BMA : West,"1,501,143,154","1,516,161,257","-15,018,104",99.01,14.69
2,202505,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527.0,G,GX7,"Central, West","1,286,244,505","1,345,256,231","-59,011,726",95.61,12.59
3,202505,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527.0,G,GX2,BMA : East,"1,747,462,726","1,832,338,419","-84,875,693",95.37,17.10
4,202505,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527.0,G,GX8,South,"1,048,914,660","1,120,019,626","-71,104,966",93.65,10.27
5,202505,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527.0,G,GX4,North,"1,133,089,170","1,239,726,001","-106,636,831",91.40,11.09
6,202505,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527.0,G,GX5,Northeast 1,"906,872,474","1,009,900,715","-103,028,241",89.80,8.88
7,202505,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527.0,G,GX6,Northeast 2,"789,107,192","908,353,043","-119,245,850",86.87,7.72


In [184]:
''' 64 Province (H) : Monthly Summary '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

h_monthly_df = raw_monthly_df.loc[raw_monthly_df['TM_KEY_MTH']==v_tm_key_mth].copy()
h_monthly_df = h_monthly_df.loc[h_monthly_df['AREA_TYPE']=='H']
# h_monthly_df = h_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)
h_monthly_df = h_monthly_df.fillna(0).sort_values(by='%ACH', ascending=False).reset_index(drop=True)

h_monthly_df_display = h_monthly_df.copy()
mod_col_list = h_monthly_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    h_monthly_df_display[col] = h_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
h_monthly_df_display['%ACH'] = h_monthly_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
h_monthly_df_display['%PORTION'] = h_monthly_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
h_monthly_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528.0,H,21Z,Rayong,"235,917,520","239,467,014","-3,549,494",98.52,2.31
1,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528.0,H,23Y,"Chanthaburi, Trat","103,609,527","106,667,004","-3,057,477",97.13,1.01
2,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528.0,H,202,"NTB : Bang Kruai, Bang Yai","114,668,543","118,062,615","-3,394,073",97.13,1.12
3,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528.0,H,007,"PTT : Mueang Pathum Thani, Lat Lum Kaeo, Sam K...","259,718,385","268,547,338","-8,828,953",96.71,2.54
4,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528.0,H,74Z,Samut Sakhon,"209,493,036","216,963,052","-7,470,016",96.56,2.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528.0,H,46X,"Kalasin, Mukdahan","95,976,592","119,200,427","-23,223,835",80.52,0.94
60,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528.0,H,44X,"Maha Sarakham, Roi Et","164,539,455","205,914,577","-41,375,121",79.91,1.61
61,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528.0,H,31Z,Buri Ram,"112,390,873","141,810,441","-29,419,568",79.25,1.10
62,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528.0,H,48X,"Sakon Nakhon, Nakhon Phanom","104,541,512","132,405,751","-27,864,239",78.96,1.02


In [185]:
''' 96 District (HH) : Monthly Summary '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

hh_monthly_df = raw_monthly_df.loc[raw_monthly_df['TM_KEY_MTH']==v_tm_key_mth].copy()
hh_monthly_df = hh_monthly_df.loc[hh_monthly_df['AREA_TYPE']=='HH']
# hh_monthly_df = hh_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)
hh_monthly_df = hh_monthly_df.fillna(0).sort_values(by='%ACH', ascending=False).reset_index(drop=True)

hh_monthly_df_display = hh_monthly_df.copy()
mod_col_list = hh_monthly_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    hh_monthly_df_display[col] = hh_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
hh_monthly_df_display['%ACH'] = hh_monthly_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
hh_monthly_df_display['%PORTION'] = hh_monthly_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
hh_monthly_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528.0,HH,902043,Samut Sakhon,"209,493,036","95,371,165","114,121,871",219.66,2.05
1,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528.0,HH,906108,Satun,"28,657,103","14,044,168","14,612,935",204.05,0.28
2,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528.0,HH,903055,Rayong,"235,917,520","132,886,546","103,030,974",177.53,2.31
3,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528.0,HH,902044,Samut Songkhram,"30,349,235","17,467,077","12,882,158",173.75,0.30
4,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528.0,HH,906106,Phuket,"189,250,646","109,782,727","79,467,919",172.39,1.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528.0,HH,909082,Nakhon Phanom,"41,773,292","44,432,642","-2,659,351",94.01,0.41
92,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528.0,HH,904068,Mae Hong Son,"14,903,229","15,918,000","-1,014,771",93.63,0.15
93,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528.0,HH,910091,Kalasin,"64,263,552","68,891,000","-4,627,448",93.28,0.63
94,202505,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528.0,HH,909087,Sakon Nakhon,"62,770,667","68,856,590","-6,085,923",91.16,0.61


### Area Insight

In [186]:
        # TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
        # , ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH, ACTUAL_AGG_QTR, ACTUAL_AGG_YR 
        # , TARGET_SNAP, TARGET_AGG_MTH, TARGET_AGG_QTR, TARGET_AGG_YR 
        # , PPN_TM